# Reference Counting

Managing reference counts of Python objects.

Reference counting is a core concept in Python's memory management system.

The objective is to gain insights into how Python handles objects in memory, reclaims unused resources and avoids memory leaks.

## What is?

- Reference counting tracks the number of `variables or references` pointing to an object in memory.
- When reference count for an object drops to zero, Python automatically frees memory occupied by that object, making it available for reuse.

## How it works?

```python
  my_var = 10
```

Here is what happens under the hood when one declares a variable in python:

- Python creates an object of type `int` with the value 10 at a specific `memory address`
- `my_var` becomes a reference (pointer) to that object.

So, at this point, in our python application, the reference coint for the object 10 is 1, because only `my_var` references it.

## Adding references

Now, adding another reference to the same object:

```python
  other_var = my_var
```

Here is what is happening:

- `other_var` doesn't create a new object or copy the value of 10. Instead, it points to the same memory address as `my_var`
- The reference count for the object 10 increases to 2.

## Removing References

When a reference is removed, the reference count decreases.

```python
  my_var = None
```

- `my_var` no longer points out to the object 10
- The reference count for the object 10 drops back to 1, so, right now, only `other_var` points to the address of the object 10.

```python
  other_var = None
```

Now, if we remove the reference to the object 10 that was being used in `other_var`, the reference counting of object 10 drops to zero.

Python's memory manager frees up the memory occupied by the object 10.

# Useful tools for inspecting Reference Counts in Python

Python provides built-in tools to inspect an object's reference count. Below, two commonly used methods.

## sys.getrefcount

`sys` module offers the `getrefcount` function, which returns the reference count of an object:

```python
import sys

my_var = [1,2,3]
print(sys.getrefcount(my_var))
```

Here is the catch when using `sys.getrefcount`. When calling this method, object's reference count is added temporarily by 1, from within the function, so the count is always at least one higher than expected.

## ctypes

For accurate counts and avoid extra reference created by `getrefcount`, one could use the ´ctypes` module. This method retrieves the reference count directly from memory:

```python

import ctypes

def ref_count(address: int) -> int:
  return ctypes.c_long.from_address(address).value

my_var = [1,2,3]
address = id(my_var)
print(ref_count(address))

```

This approach provides an accurate count without the additional reference introduced bt `sys.getrefcount`

In [ ]:
import ctypes

def ref_count(address: int) -> int:
  """ 
  function to get the reference count of an object at a given memory address 
  
  input:
    address: int - memory address of the object
    
  returns:
    int - reference count of the object
  """
  
  return ctypes.c_long.from_address(address).value

a = [1, 2, 3]
b = a
c = a

print(f"Reference count of 'a': {ref_count(id(a))}")

b = None

print(f"Reference count of 'a' after setting 'b' to None: {ref_count(id(a))}")

c = None

print(f"Reference count of 'a' after setting 'c' to None: {ref_count(id(a))}")

# Objects out of Scope

When variables go out of scope (after a function call), their references are automatically removed:

```python
def create_object():
  obj = [1,2,3]
  print(ref_count(id(obj)))

create_object()
```

# Why reference Counting matters

Reference counting is the foundation of Python's memory management.

- Efficient memory usage: By tracking references, Python ensures that memory is freed when objects are no longer needed
- Garbage collection support: It works alongside Python's garbage collector, which handles more complex memory scenarios
- Debugging and Optimization: These tools can help on debugging memory issues and optimze code

# Common Pitfalls and Best Practices

- Misinterpreting `sys.getrefcount`: forgetting that when using this method, you need to subtract one to the returned value
- Avoid Manual Memory Management: Python handles memory, so, working with memory addresses is rarely necessary
- Beware of Cyclic References: Reference counting alone cannot handle cyclic references

```python
a = []
a.append(a)

```

`Garbage Collector` handles cases of cyclic references.